In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.0/540.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.3
    Uninstalling typer-0.15.3:
      Successfully uninstalled typer-0.15.3
ERROR: pip's dependency resolver

In [ ]:
!pip install -U "flwr[simulation]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 34.7 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/
!ls *.py

/content/drive/MyDrive/Colab Notebooks
client.py  cnn_models.py  dataset.py  main.py  model.py  server.py


In [ ]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

GPU available: True
Device name: NVIDIA A100-SXM4-40GB


# model.py

In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/model.py"

import torch
import torch.nn as nn
import torchvision.models as models

# Helper to auto-detect flattened size
class FlattenedSizeDetector(nn.Module):
    def __init__(self, feature_extractor):
        super().__init__()
        self.features = feature_extractor

    def forward(self, x):
        with torch.no_grad():
            x = self.features(x)
            return x.view(x.size(0), -1).size(1)

# ------------------------------
# 1. TinyCNN (Input: 128x128)
# ------------------------------
class TinyCNN(nn.Module):
    def __init__(self, num_classes=3):
        super(TinyCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 8, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # [8, 64, 64]
        )
        # Auto-detect flattened size
        dummy_input = torch.zeros(1, 3, 128, 128)
        flat_size = self.features(dummy_input).view(1, -1).size(1)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat_size, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# ------------------------------
# 2. SimpleCNN_V2 (Input: 128x128)
# ------------------------------
class SimpleCNN_V2(nn.Module):
    def __init__(self, num_classes=3):
        super(SimpleCNN_V2, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),  # -> [32, 64, 64]
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),  # -> [64, 32, 32]
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))  # -> [128, 1, 1]
        )

        dummy_input = torch.zeros(1, 3, 128, 128)
        flat_size = self.features(dummy_input).view(1, -1).size(1)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.4),
            nn.Linear(flat_size, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

# ------------------------------
# 3. ResNet-18
# ------------------------------
class ResNet18(nn.Module):
    def __init__(self, num_classes=3):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# ------------------------------
# 4. MobileNetV2
# ------------------------------
class MobileNetV2(nn.Module):
    def __init__(self, num_classes=3):
        super(MobileNetV2, self).__init__()
        self.model = models.mobilenet_v2(pretrained=False)
        self.model.classifier[1] = nn.Linear(self.model.last_channel, num_classes)

    def forward(self, x):
        return self.model(x)

Overwriting /content/drive/MyDrive/Colab Notebooks/model.py


# dataset.py

In [ ]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/dataset.py"

import os
import shutil
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

def get_dataloaders(data_dir, batch_size=32, val_split=0.2, seed=42):

    for class_dir in os.listdir(data_dir):
        full_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(full_path) and len(os.listdir(full_path)) == 0:
            print(f" Removing empty folder: {full_path}")
            os.rmdir(full_path)

    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

    val_size = int(val_split * len(full_dataset))
    train_size = len(full_dataset) - val_size

    torch.manual_seed(seed)
    train_ds, val_ds = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)

    return train_loader, val_loader

Overwriting /content/drive/MyDrive/Colab Notebooks/dataset.py


# server.py

In [ ]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/server.py"

import flwr as fl

def get_strategy():
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=3,
    )
    return strategy

Overwriting /content/drive/MyDrive/Colab Notebooks/server.py


# client.py

In [ ]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/client.py"

import flwr as fl
from flwr.common import parameters_to_ndarrays, ndarrays_to_parameters
from flwr.client import NumPyClient
import torch
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import csv

sys.path.append('/content/drive/MyDrive/Colab Notebooks')
from dataset import get_dataloaders
from model import TinyCNN, SimpleCNN_V2, ResNet18, MobileNetV2

LOG_DIR = "/content/drive/MyDrive/Colab Notebooks/logs"
USE_GPU = os.environ.get("USE_GPU", "false").lower() == "true"
DEVICE = torch.device("cuda" if USE_GPU and torch.cuda.is_available() else "cpu")

def get_model(model_name, num_classes=3):
    if model_name == "tinycnn":
        return TinyCNN(num_classes).to(DEVICE)
    elif model_name == "simplecnn":
        return SimpleCNN_V2(num_classes).to(DEVICE)
    elif model_name == "resnet18":
        return ResNet18(num_classes).to(DEVICE)
    elif model_name == "mobilenetv2":
        return MobileNetV2(num_classes).to(DEVICE)
    else:
        raise ValueError(f"Unknown model name: {model_name}")

class FlowerClient(NumPyClient):
    def __init__(self, model_name, data_path, client_id, split_name="default"):
        self.model = get_model(model_name)
        self.train_loader, self.val_loader = get_dataloaders(data_path)
        self.client_id = client_id
        self.split_name = split_name

        # Initialize log file
        os.makedirs(LOG_DIR, exist_ok=True)

        # Initialize log file
        self.log_file = f"{LOG_DIR}/client_{client_id}_metrics_{model_name}_{split_name}.csv"
        if not os.path.exists(self.log_file):
            with open(self.log_file, "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerow(["round", "loss", "accuracy"])

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters, config=None):
        # Convert if needed
        if hasattr(parameters, "tensors"):  # it's a Parameters object
            parameters = parameters_to_ndarrays(parameters)

        # Load into state_dict
        state_dict = self.model.state_dict()
        for k, val in zip(state_dict.keys(), parameters):
            state_dict[k] = torch.tensor(val)
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters, config)
        self.model.train()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        for epoch in range(1):
            for x, y in self.train_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                optimizer.zero_grad()
                y_hat = self.model(x)
                loss = F.cross_entropy(y_hat, y)
                loss.backward()
                optimizer.step()

        return self.get_parameters(config), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters, config)
        self.model.eval()

        correct = 0
        loss = 0.0
        total = 0

        with torch.no_grad():
            for x, y in self.val_loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                y_hat = self.model(x)
                loss += F.cross_entropy(y_hat, y, reduction='sum').item()
                correct += (y_hat.argmax(1) == y).sum().item()
                total += y.size(0)

        avg_loss = loss / total
        accuracy = correct / total
        round_number = config.get("server_round", 0)

        # Log results to CSV
        if config.get("server_side", False):
            with open(self.log_file, "a", newline="") as f:
                writer = csv.writer(f)
                writer.writerow([round_number, avg_loss, accuracy])

        return avg_loss, total, {"accuracy": accuracy}

Overwriting /content/drive/MyDrive/Colab Notebooks/client.py


# main.py

In [ ]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/main.py"

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

import os
import csv
import flwr as fl
from flwr.common import Context
from flwr.simulation import start_simulation
from client import FlowerClient

# Constants
LOG_DIR = "/content/drive/MyDrive/Colab Notebooks/logs"
DATA_BASE = "/content/drive/MyDrive/Colab Notebooks/Covid19-dataset/splits"
CLIENTS = []

os.makedirs(LOG_DIR, exist_ok=True)

# ------------------------------
# Evaluation Function
# ------------------------------
def get_eval_fn(model_name, split_name):
    def evaluate(server_round, parameters, config):
        total_correct = 0
        total_samples = 0
        round_logs = []

        for i, client in enumerate(CLIENTS):
            # print(f"Client {i} evaluate arg count:", client.evaluate.__code__.co_argcount)
            loss, num_samples, metrics = client.evaluate(parameters, config)
            acc = metrics.get("accuracy", 0.0)
            total_correct += acc * num_samples
            total_samples += num_samples
            round_logs.append([server_round, i, acc, loss])

        # Log client metrics
        log_client_path = f"{LOG_DIR}/log_client_metrics_{model_name}_{split_name}.csv"
        with open(log_client_path, "a", newline="") as f:
            writer = csv.writer(f)
            if os.stat(log_client_path).st_size == 0:
                writer.writerow(["round", "client_id", "accuracy", "loss"])
            writer.writerows(round_logs)

        # Log global metrics
        global_acc = total_correct / total_samples if total_samples > 0 else 0.0
        log_global_path = f"{LOG_DIR}/log_global_metrics_{model_name}_{split_name}.csv"
        with open(log_global_path, "a", newline="") as f:
            writer = csv.writer(f)
            if os.stat(log_global_path).st_size == 0:
                writer.writerow(["round", "global_accuracy"])
            writer.writerow([server_round, global_acc])

        return 0.0, {"accuracy": global_acc}

    return evaluate

# ------------------------------
# Custom Strategy Subclass
# ------------------------------
class CustomFedAvg(fl.server.strategy.FedAvg):
    def __init__(self, eval_fn, **kwargs):
        super().__init__(**kwargs)
        self._eval_fn_custom = eval_fn

    def evaluate(self, server_round, parameters, config=None):  # Make config optional
        if config is None:
            config = {}  # Provide empty config if missing
        config["server_round"] = server_round
        config["server_side"] = True
        return self._eval_fn_custom(server_round, parameters, config)

# ------------------------------
# Client Setup
# ------------------------------
def get_clients(model_name, split_name):
    global CLIENTS
    clients = []
    for i in range(3):
        data_path = os.path.join(DATA_BASE, split_name, f"Client-{i+1}")
        # print(f"✅ Initialized client {i} with path: {data_path}")
        client = FlowerClient(model_name, data_path, i, split_name)
        clients.append(client)
    CLIENTS = clients
    return clients

# ------------------------------
# Run Simulation
# ------------------------------
def run_simulation(model_name, split_name):
    clients = get_clients(model_name, split_name)
    eval_fn = get_eval_fn(model_name, split_name)

    strategy = CustomFedAvg(
        eval_fn=eval_fn,
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=3,
        min_evaluate_clients=3,
        min_available_clients=3
    )

    def client_fn(cid: str):
      return clients[int(cid)]

    start_simulation(
        client_fn=client_fn,
        num_clients=3,
        config=fl.server.ServerConfig(num_rounds=10),
        strategy=strategy
    )

# ------------------------------
# Entrypoint
# ------------------------------
if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python main.py <model_name> <split_name>")
        sys.exit(1)

    model = sys.argv[1].lower()
    split = sys.argv[2]
    run_simulation(model, split)

Overwriting /content/drive/MyDrive/Colab Notebooks/main.py


# Runner code

In [ ]:
import subprocess
from tqdm import tqdm

models = ["tinycnn", "simplecnn", "resnet18", "mobilenetv2"]
splits = ["Label_skew", "Concept_shift", "Feature_skew", "Pathological", "Quantity_skew", "IID_equal", "Dirichlet_label"]

# Total runs = len(models) × len(splits)
total_runs = len(models) * len(splits)

with tqdm(total=total_runs, desc="Running experiments") as pbar:
    for split in splits:
        for model in models:
            tqdm.write(f"\n▶️  Running: {model} on {split}")
            subprocess.run(["python", "main.py", model, split])
            pbar.update(1)

Running experiments:   0%|          | 0/28 [00:00<?, ?it/s]


▶️  Running: tinycnn on Label_skew


Running experiments:   4%|▎         | 1/28 [07:14<3:15:23, 434.19s/it]


▶️  Running: simplecnn on Label_skew


Running experiments:   7%|▋         | 2/28 [16:38<3:41:16, 510.63s/it]

...

Running experiments:  93%|█████████▎| 26/28 [4:09:54<24:49, 744.88s/it]


▶️  Running: resnet18 on Dirichlet_label


Running experiments:  96%|█████████▋| 27/28 [4:24:49<13:09, 789.90s/it]


▶️  Running: mobilenetv2 on Dirichlet_label


Running experiments: 100%|██████████| 28/28 [4:38:34<00:00, 596.94s/it]


# Label_skew

In [ ]:
!python main.py tinycnn Label_skew

2025-06-01 14:51:28.714439: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748789488.735681    9220 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748789488.742127    9220 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-01 14:51:28.763369: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for exa

In [ ]:
!python main.py simplecnn Label_skew

In [ ]:
!python main.py resnet18 Label_skew

In [ ]:
!python main.py mobilenetv2 Label_skew

# Concept_shift

In [ ]:
!python main.py tinycnn Concept_shift

In [ ]:
!python main.py simplecnn Concept_shift

In [ ]:
!python main.py resnet18 Concept_shift

In [ ]:
!python main.py mobilenetv2 Concept_shift

# Feature_skew

In [ ]:
!python main.py tinycnn Feature_skew

In [ ]:
!python main.py simplecnn Feature_skew

In [ ]:
!python main.py resnet18 Feature_skew

In [ ]:
!python main.py mobilenetv2 Feature_skew

# Pathological

In [ ]:
!python main.py tinycnn Pathological

2025-06-02 16:47:22.667002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748882842.688243   10026 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748882842.694690   10026 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 16:47:22.716685: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
 Removing empty folder: /content/drive/MyDrive/Colab Notebooks/Covid19-dataset/splits/Pathological/Client

In [ ]:
!python main.py simplecnn Pathological

2025-06-02 16:59:31.686968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748883571.708545   16215 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748883571.715216   16215 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 16:59:31.737107: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for exa

In [ ]:
!python main.py resnet18 Pathological

2025-06-02 17:13:30.003172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748884410.025438   21338 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748884410.032129   21338 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 17:13:30.053915: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pre

In [ ]:
!python main.py mobilenetv2 Pathological

2025-06-02 17:30:24.970775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748885424.992287   27088 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748885424.998791   27088 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 17:30:25.020011: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pre

# Dirichlet_label

In [ ]:
!python main.py tinycnn Dirichlet_label

In [ ]:
!python main.py tinycnn Dirichlet_label

In [ ]:
!python main.py tinycnn Dirichlet_label

In [ ]:
!python main.py tinycnn Dirichlet_label

# Quantity_skew

In [ ]:
!python main.py tinycnn Quantity_skew

In [ ]:
!python main.py tinycnn Quantity_skew

In [ ]:
!python main.py tinycnn Quantity_skew

In [ ]:
!python main.py tinycnn Quantity_skew

# IID_equal

In [ ]:
!python main.py tinycnn IID_equal

In [ ]:
!python main.py tinycnn IID_equal

In [ ]:
!python main.py tinycnn IID_equal

In [ ]:
!python main.py tinycnn IID_equal